In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:\cygwin64\home\salis\data\caesarian.csv')

In [3]:
df.rename(columns={" caesarian": "caesarian", " delivery number": "delivery_number",
                  " delivery time":"delivery_time", " blood pressure":"blood_pressure",
                  " heart problem":"heart_problem"}, inplace=True)

In [4]:
df.columns

Index(['age', 'delivery_number', 'delivery_time', 'blood_pressure',
       'heart_problem', 'caesarian'],
      dtype='object')

In [6]:
df.head()

,age,delivery_number,delivery_time,blood_pressure,heart_problem,caesarian
0,22,1,0,2,0,0
1,26,2,0,1,0,1
2,26,2,1,1,0,0
3,28,1,0,2,0,0
4,22,2,0,1,0,1


In [7]:
C = len(df['caesarian'].unique())

In [8]:
df.groupby(['caesarian','heart_problem']).size().reset_index(name='counts')

,caesarian,heart_problem,counts
0,0,0,28
1,0,1,6
2,1,0,22
3,1,1,24


In [9]:
df.groupby(['caesarian','delivery_time']).size().reset_index(name='counts')

,caesarian,delivery_time,counts
0,0,0,16
1,0,1,9
2,0,2,9
3,1,0,30
4,1,1,8
5,1,2,8


In [13]:
def evalNumerAttrib(feat):
    # hold my split info
    splits = []
    
    # sort on current column
    C = np.sort(feat.unique())
    
    # sort attribute
    X = np.sort(feat)    
    
    # create set to hold midpoints
    midpoints = set()
    
    # loop through classes
    for i,val in enumerate(C):
        if i+1 == len(C):
            continue
        else:
            # add midpoint to set
            midpoints.add((val + C[i+1])/2)
        
    for mp in midpoints:
        # get probability for current midpoint
        yes_prob = feat.where(feat < mp).count() / feat.count()
        no_prob = feat.where(feat > mp).count() / feat.count()
        gi = (yes_prob * (1 - yes_prob)) + (no_prob * (1 - no_prob))
        
        splits.append({'mp':mp, 'gi':gi, 'samples': feat.count(),
            'values':[feat.where(feat < mp).count(),feat.where(feat > mp).count()]})
        
    return splits
    

In [19]:
for column in df.drop(['caesarian','age'], axis=1):
    # for the first level, check each column
    maxGI = max(evalNumerAttrib(df[column]), key=lambda x:x['gi'])
    print(column, maxGI)
    
    

delivery_number {'mp': 1.5, 'gi': 0.49968749999999995, 'samples': 80, 'values': [41, 39]}
delivery_time {'mp': 0.5, 'gi': 0.48875, 'samples': 80, 'values': [46, 34]}
blood_pressure {'mp': 0.5, 'gi': 0.375, 'samples': 80, 'values': [20, 60]}
heart_problem {'mp': 0.5, 'gi': 0.46875, 'samples': 80, 'values': [50, 30]}


In [18]:
for column in df.drop(['caesarian','age','delivery_number'], axis=1):
    # for the first level, check each column
    maxGI = max(evalNumerAttrib(df[df['delivery_number']<=1.5][column]), key=lambda x:x['gi'])
    print(column, maxGI)
    print('----')
    maxGI = max(evalNumerAttrib(df[df['delivery_number']>=1.5][column]), key=lambda x:x['gi'])
    print(column, maxGI)
    print("\n")


delivery_time {'mp': 0.5, 'gi': 0.49732302201070794, 'samples': 41, 'values': [22, 19]}
----
delivery_time {'mp': 0.5, 'gi': 0.47337278106508873, 'samples': 39, 'values': [24, 15]}


blood_pressure {'mp': 0.5, 'gi': 0.44973230220107074, 'samples': 41, 'values': [14, 27]}
----
blood_pressure {'mp': 1.5, 'gi': 0.3261012491781723, 'samples': 39, 'values': [31, 8]}


heart_problem {'mp': 0.5, 'gi': 0.4140392623438429, 'samples': 41, 'values': [29, 12]}
----
heart_problem {'mp': 0.5, 'gi': 0.4970414201183432, 'samples': 39, 'values': [21, 18]}


